<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/T5/t5_without_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 367, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 367 (delta 47), reused 6 (delta 3), pack-reused 265
Receiving objects: 100% (367/367), 181.43 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (160/160), done.
Updating files: 100% (26/26), done.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/RuLawSimplification/corpus/full_corpus.zip')

In [ ]:
df.head(2)

,Название документа,Ссылка,Текст,Комментарий РГ,Date
0,Постановление Конституционного Суда Российской...,https://rg.ru/documents/2022/11/25/document-ob...,Именем Российской Федерации\n Конституционный...,"Бессрочный запрет на профессию для адвокатов, ...",2022-11-25
1,Постановление Пленума Верховного Суда Российск...,https://rg.ru/documents/2022/11/25/document-pl...,В целях обеспечения единства практики применен...,Пленум Верховного суда России утвердил постано...,2022-11-25


In [ ]:
data = df.copy()

df = df.drop(['Название документа', 'Ссылка', 'Date'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
train.to_csv('train.csv', index=False, header=None)
test.iloc[:200].to_csv('validation.csv', index=False, header=None)
test.iloc[200:].to_csv('test.csv', index=False, header=None)

In [ ]:
!pip install Sentencepiece
!pip install transformers datasets accelerate nvidia-ml-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!nvidia-smi

Sat Mar 11 08:08:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    54W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from datasets import load_dataset


dataset = load_dataset('csv', 
                       data_files={'train': ['train.csv'],
                                   'validation': ['validation.csv'],
                                   'test': ['test.csv']},
                       encoding='utf8',
                       index_col = False,
                       header=None,
                       sep=','
                      )

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4ce8e812e15da9d2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
pd.read_csv('train.csv')

In [ ]:
print(dataset['train'])
print(dataset['validation'])
print(dataset['test'])

Dataset({
    features: ['0', '1'],
    num_rows: 1621
})
Dataset({
    features: ['0', '1'],
    num_rows: 200
})
Dataset({
    features: ['0', '1'],
    num_rows: 206
})


In [ ]:
dataset.shape

{'train': (1621, 2), 'validation': (200, 2), 'test': (206, 2)}

In [ ]:
dataset.push_to_hub('marcus2000/legal_simplification_corpus')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset('marcus2000/legal_simplification_corpus')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1621 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/206 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/marcus2000___parquet/marcus2000--legal_simplification_corpus-6d810038b9adc228/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset.set_format("pt")

In [ ]:
dataset['train']

Dataset({
    features: ['0', '1'],
    num_rows: 1621
})

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"], 2)

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig


tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruT5-base", use_fast=True)



In [ ]:
max_input_length = 20000
max_target_length = 1000


def preprocess_function(examples):
    inputs = [ex for ex in examples['0']]  # 0 - Текст
    targets = [ex for ex in examples['1']]  # 1 - Комментарий
    model_inputs = tokenizer(inputs, 
                             max_length=max_input_length,
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1621 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 33014
    })
    test: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4254
    })
    validation: Dataset({
        features: ['0', '1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_hf = AutoModelForSeq2SeqLM.from_pretrained("sberbank-ai/ruT5-base")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 1

args = Seq2SeqTrainingArguments("legal_text_simplifier",
    evaluation_strategy = "epoch",
    learning_rate=5e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    push_to_hub=True,
    predict_with_generate=True,
    fp16=True
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_hf)

In [ ]:
!pip install rouge_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=6e9a02351254032ca278e8dd90ca4438c3abeaaa2bf5e2a11ca75bf9b7d4ccec
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [ ]:
!pip install sacremoses
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7d8c1c7c1510a1159c05e854ee9370d8619086e607484c3b61fd59d68fdf0d76
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 4.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric


sari = load_metric("sari")

<ipython-input-16-46b18dc41a4f>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  sari = load_metric("sari")


In [ ]:
rouge = load_metric("rouge")

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels



In [ ]:
def compute_metrics(eval_preds):
    inputs, preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_inputs, decoded_preds, decoded_labels = postprocess_text(decoded_inputs, decoded_preds,
                                                                        decoded_labels, "sari")
    sari_result = sari._compute(inputs=decoded_inputs, predictions=decoded_preds, references=decoded_labels)

In [ ]:
trainer = Seq2SeqTrainer(
    model_hf,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

Cloning https://huggingface.co/marcus2000/legal_text_simplifier into local empty directory.


Download file pytorch_model.bin:   0%|          | 17.4k/2.75G [00:00<?, ?B/s]

Download file runs/Feb25_16-44-49_afb95afb5944/events.out.tfevents.1677344304.afb95afb5944.116.0: 100%|#######…

Download file runs/Feb23_19-49-53_63556cc90794/1677181959.6084156/events.out.tfevents.1677181959.63556cc90794.…

Clean file runs/Feb25_16-44-49_afb95afb5944/events.out.tfevents.1677344304.afb95afb5944.116.0:  13%|#2        …

...

Clean file runs/Feb23_19-59-26_63556cc90794/events.out.tfevents.1677182385.63556cc90794.1560.7:  21%|##1      …

Clean file training_args.bin:  28%|##8       | 1.00k/3.56k [00:00<?, ?B/s]

Clean file runs/Feb23_19-49-53_63556cc90794/events.out.tfevents.1677181959.63556cc90794.1560.5:  21%|##1      …

Clean file pytorch_model.bin:   0%|          | 1.00k/2.75G [00:00<?, ?B/s]

Using cuda_amp half precision backend


In [ ]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

print_gpu_utilization()

GPU memory occupied: 3861 MB.


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 33014
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 165070
  Number of trainable parameters = 737668096
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to en

Epoch,Training Loss,Validation Loss


Saving model checkpoint to legal_text_simplifier/checkpoint-500
Configuration saved in legal_text_simplifier/checkpoint-500/config.json
Configuration saved in legal_text_simplifier/checkpoint-500/generation_config.json
Model weights saved in legal_text_simplifier/checkpoint-500/pytorch_model.bin
tokenizer config file saved in legal_text_simplifier/checkpoint-500/tokenizer_config.json
Special tokens file saved in legal_text_simplifier/checkpoint-500/special_tokens_map.json
Copy vocab file to legal_text_simplifier/checkpoint-500/spiece.model
tokenizer config file saved in legal_text_simplifier/tokenizer_config.json
Special tokens file saved in legal_text_simplifier/special_tokens_map.json
Copy vocab file to legal_text_simplifier/spiece.model
Saving model checkpoint to legal_text_simplifier/checkpoint-1000
Configuration saved in legal_text_simplifier/checkpoint-1000/config.json
Configuration saved in legal_text_simplifier/checkpoint-1000/generation_config.json
Model weights saved in legal

In [ ]:
predidcted = trainer.predict(tokenized_datasets['test'])

The following columns in the test set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: 1, 0. If 1, 0 are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 492
  Batch size = 16


In [ ]:
predidcted

PredictionOutput(predictions=array([[     0, 250099,    260, ...,      0,      0,      0],
       [     0, 250099,    259, ...,      0,      0,      0],
       [     0, 250099,    260, ...,      0,      0,      0],
       ...,
       [     0, 250099,    259, ...,      0,      0,      0],
       [     0, 250099,    259, ...,      0,      0,      0],
       [     0, 250099,    260, ...,      0,      0,      0]]), label_ids=array([[  3835,  54312,   2007, ...,   -100,   -100,   -100],
       [   259,   6835,   3262, ...,   -100,   -100,   -100],
       [  1908,  56364,   6203, ...,   -100,   -100,   -100],
       ...,
       [   259, 109415,  63690, ...,   -100,   -100,   -100],
       [   688,  13890,    259, ...,   -100,   -100,   -100],
       [  3771,   3262,   2527, ...,   -100,   -100,   -100]]), metrics={'test_loss': nan, 'test_bleu': 0.0939, 'test_gen_len': 7.435, 'test_runtime': 15.543, 'test_samples_per_second': 31.654, 'test_steps_per_second': 1.994})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to multilingual_t5_model_for_law_simplification
Configuration saved in multilingual_t5_model_for_law_simplification/config.json
Model weights saved in multilingual_t5_model_for_law_simplification/pytorch_model.bin
tokenizer config file saved in multilingual_t5_model_for_law_simplification/tokenizer_config.json
Special tokens file saved in multilingual_t5_model_for_law_simplification/special_tokens_map.json
Copy vocab file to multilingual_t5_model_for_law_simplification/spiece.model
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/2.17G [00:00<?, ?B/s]

Upload file runs/Sep15_19-55-46_0ab1f4eeae4b/events.out.tfevents.1663271772.0ab1f4eeae4b.70.0:  40%|###9      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification
   b4a4ac6..bca9218  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification
   b4a4ac6..bca9218  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Bleu', 'type': 'bleu', 'value': 0.0939}]}
To https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification
   bca9218..c5eb755  main -> main

   bca9218..c5eb755  main -> main



'https://huggingface.co/marcus2000/multilingual_t5_model_for_law_simplification/commit/bca9218d4ff16dcd8b849938a8a8bf498d5c4806'

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 7.0 MB/s 


In [ ]:
import evaluate
# evaluate.list_evaluation_modules(module_type='metric', with_details=True)

In [ ]:
metric_value = 8.1  # value obtained from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_conformer_ctc_small 

evaluate.push_to_hub(
    model_id='marcus2000/fine_tuned_t5_model',
    task_type="automatic-speech-recognition",
    dataset_type="librispeech_asr",
    dataset_name="Librispeech (clean)",
    metric_type="wer",
    metric_name="WER",
    dataset_split="test",  # corresponds to test-clean set
    dataset_config="other",  # corresponds to test-clean set
    dataset_args=dict(language="en"),  # metadata for dataset
    # the actual score obtained by the model
    metric_value=metric_value,
)

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

'https://huggingface.co/marcus2000/fine_tuned_t5_model/blob/main/README.md'